In [8]:
import pandas as pd
from xgboost import XGBClassifier

# Load the model with the same parameters as in your training code
best_params = {'learning_rate': 0.16640983692081657, 'n_estimators': 70, 'max_depth': 10, 'min_child_weight': 1, 'subsample': 0.9627772663131126, 'colsample_bytree': 0.8328125240505475, 'gamma': 0.13209771833800843, 'reg_alpha': 0.7012278052521271, 'reg_lambda': 0.944583621847628, 'scale_pos_weight': 3.2875411136098283}
model = XGBClassifier(**best_params, random_state=42, eval_metric='logloss')

# Define the features in the same order as used in training
features = [
    'PlayoffProgScore', '3P%', 'FT%',
    'OFFRTG', 'DEFRTG', 'AST RATIO',
    'REB%', 'TOV%', 'PACE', 'AvgPIE_NextYearPlayers'
]

# Load and prepare the training data
dataset = pd.read_csv('./cleaned_data/dataset.csv')
X_train = dataset[features]
y_train = dataset['PlayoffNextSeason']

# Train the model on all training data
model.fit(X_train, y_train)

# Load year 10 data
year_10_data = pd.read_csv('./cleaned_data/year_10_stats.csv')

# Prepare the prediction data
X_pred = year_10_data[features]

# Get predictions and probabilities
predictions = model.predict(X_pred)
probabilities = model.predict_proba(X_pred)

# Create results DataFrame
results = pd.DataFrame({
    'Team': year_10_data['Team'],
    'Playoff_Prediction': predictions,
    'Playoff_Probability': probabilities[:, 1]  # Probability of positive class (making playoffs)
})

# Sort by probability in descending order
results = results.sort_values('Playoff_Probability', ascending=False)

# Display results
print("\nPlayoff Predictions for Next Season:")
print("=====================================")
for _, row in results.iterrows():
    print(f"{row['Team']}: {row['Playoff_Probability']*100:.2f}% chance of making playoffs")

# Save to CSV
results.to_csv('playoff_predictions_year_10.csv', index=False)
print("\nResults have been saved to 'playoff_predictions_year_10.csv'")


Playoff Predictions for Next Season:
LAS: 99.63% chance of making playoffs
IND: 98.37% chance of making playoffs
SAS: 96.35% chance of making playoffs
MIN: 95.19% chance of making playoffs
SAC: 95.18% chance of making playoffs
DET: 93.56% chance of making playoffs
PHO: 71.79% chance of making playoffs
SEA: 69.19% chance of making playoffs
WAS: 57.46% chance of making playoffs
ATL: 50.71% chance of making playoffs
NYL: 23.21% chance of making playoffs
CON: 15.91% chance of making playoffs
CHI: 5.61% chance of making playoffs
TUL: 5.10% chance of making playoffs

Results have been saved to 'playoff_predictions_year_10.csv'
